In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datetime import datetime
import numpy as np
import os
import pandas as pd

path = '/content/drive/My Drive/DAAN 570'
os.chdir(path)

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np

# Load and preprocess the DataFrame
df = pd.read_csv("release_10_23_2020.csv")
df['server_timestamp'] = pd.to_datetime(df['server_timestamp_epoch_ms'], unit='ms', origin='unix')
df.sort_values(by=['session_id_hash', 'server_timestamp'], inplace=True)
df.drop('server_timestamp_epoch_ms', axis=1, inplace=True)
df['duration_minutes'] = df.groupby('session_id_hash')['server_timestamp'].diff().dt.total_seconds().div(60).fillna(0)
df['hour'] = df['server_timestamp'].dt.hour
df['day_of_month'] = df['server_timestamp'].dt.day
df['week_of_month'] = np.ceil(df['day_of_month'] / 7).astype(int)
df['date'] = df['server_timestamp'].dt.date

# Create session-level summary
from collections import defaultdict
data_session = defaultdict(dict)
for _, row in df.iterrows():
    session_id_hash = row['session_id_hash']
    event_type = row['event_type']
    data_session[session_id_hash][event_type] = data_session[session_id_hash].get(event_type, 0) + 1
    product_action = row['product_action']
    if not pd.isnull(product_action):
        data_session[session_id_hash][product_action] = data_session[session_id_hash].get(product_action, 0) + 1
        product_url = row["product_skus_hash"]
        if "seen product url" not in data_session[session_id_hash]:
            data_session[session_id_hash]["seen product url"] = [product_url]
        else:
            if product_url not in data_session[session_id_hash]["seen product url"]:
                data_session[session_id_hash]["seen product url"].append(product_url)
df_session = pd.DataFrame.from_dict(data_session, orient='index').fillna(0)
df_session.reset_index(inplace=True)
df_session.rename(columns={'index': 'session_id_hash'}, inplace=True)

# Prepare product-related DataFrame
df_product = df.dropna(subset=['product_skus_hash']).groupby(
    ['session_id_hash', 'product_skus_hash', 'event_type', 'product_action']
).agg(product_event_number=('event_type', 'count'), product_action_number=('product_action', 'count')).reset_index().set_index('session_id_hash')

# Pivot table for product stats
df_product_stats = pd.pivot_table(df, index=['session_id_hash', 'product_skus_hash'], columns=['event_type', 'product_action'],
                                  aggfunc={'product_action': 'count'}, fill_value=0)
df_product_stats.columns = ["_".join([str(a) for a in s]) for s in df_product_stats.columns.to_list()]
df_product_stats.reset_index(inplace=True)
df_product_stats.set_index("session_id_hash", inplace=True)

# Time data creation
new_time_data = df.groupby(['session_id_hash', 'product_skus_hash']).agg(
    first_interaction=pd.NamedAgg(column='server_timestamp', aggfunc='min'),
    last_interaction=pd.NamedAgg(column='server_timestamp', aggfunc='max')
).reset_index()
new_time_data['engagement_duration'] = (new_time_data['last_interaction'] - new_time_data['first_interaction']).dt.total_seconds()
new_time_data.set_index(['session_id_hash', 'product_skus_hash'], inplace=True)

# Join operations
# Ensure both DataFrames have the necessary columns in their structure, not as an index
df_combined = df_session.set_index("session_id_hash").join(df_product_stats, how="inner", rsuffix="_product")
df_combined_reset = df_combined.reset_index()
new_time_data_reset = new_time_data.reset_index()

# Use merge() to join on both 'session_id_hash' and 'product_skus_hash'
df_final = pd.merge(df_combined_reset, new_time_data_reset, on=['session_id_hash', 'product_skus_hash'], how='inner')

# The rest of your operations
df_final['number_of_seen_url'] = df_final['seen product url'].apply(lambda x: len(np.unique(x)))
df_final = df_final.dropna(subset=['detail'])

print(df_final.head())


print(df_final.head())


                            session_id_hash  pageview  detail  \
0  00007d15aeb741b3cdd873cb3933351d699cc320       7.0     3.0   
1  00007d15aeb741b3cdd873cb3933351d699cc320       7.0     3.0   
2  00007d15aeb741b3cdd873cb3933351d699cc320       7.0     3.0   
3  00014c4428e61aca7e64be1064d2b6cf49233cfb      14.0     2.0   
4  00014c4428e61aca7e64be1064d2b6cf49233cfb      14.0     2.0   

                                    seen product url  event  click  add  \
0  [90add4dd072e160034e98ddea257e0b59441eae00d895...    0.0    0.0  0.0   
1  [90add4dd072e160034e98ddea257e0b59441eae00d895...    0.0    0.0  0.0   
2  [90add4dd072e160034e98ddea257e0b59441eae00d895...    0.0    0.0  0.0   
3  [1206216ad33f70eacc1036ac8690a1857168e9fd03512...    2.0    1.0  0.0   
4  [1206216ad33f70eacc1036ac8690a1857168e9fd03512...    2.0    1.0  0.0   

   purchase  remove                                  product_skus_hash  \
0       0.0     0.0  90add4dd072e160034e98ddea257e0b59441eae00d8955...   
1       0.

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520732 entries, 0 to 1520731
Data columns (total 19 columns):
 #   Column                            Non-Null Count    Dtype         
---  ------                            --------------    -----         
 0   session_id_hash                   1520732 non-null  object        
 1   pageview                          1520732 non-null  float64       
 2   detail                            1520732 non-null  float64       
 3   seen product url                  1520732 non-null  object        
 4   event                             1520732 non-null  float64       
 5   click                             1520732 non-null  float64       
 6   add                               1520732 non-null  float64       
 7   purchase                          1520732 non-null  float64       
 8   remove                            1520732 non-null  float64       
 9   product_skus_hash                 1520732 non-null  object        
 10  product_action_eve

In [ ]:
df_final.head(5)

,session_id_hash,pageview,detail,seen product url,event,click,add,purchase,remove,product_skus_hash,product_action_event_add,product_action_event_click,product_action_event_remove,product_action_pageview_detail,product_action_pageview_purchase,first_interaction,last_interaction,engagement_duration,number_of_seen_url
0,00007d15aeb741b3cdd873cb3933351d699cc320,7.0,3.0,[90add4dd072e160034e98ddea257e0b59441eae00d895...,0.0,0.0,0.0,0.0,0.0,90add4dd072e160034e98ddea257e0b59441eae00d8955...,0,0,0,1,0,2018-12-15 21:23:55.879,2018-12-15 21:23:55.879,0.000,3
1,00007d15aeb741b3cdd873cb3933351d699cc320,7.0,3.0,[90add4dd072e160034e98ddea257e0b59441eae00d895...,0.0,0.0,0.0,0.0,0.0,9fe0ebc298344bafad714091892d02a5a501f1bffe9fa5...,0,0,0,1,0,2018-12-15 21:28:54.734,2018-12-15 21:28:54.734,0.000,3
2,00007d15aeb741b3cdd873cb3933351d699cc320,7.0,3.0,[90add4dd072e160034e98ddea257e0b59441eae00d895...,0.0,0.0,0.0,0.0,0.0,bd81fd361f0ae5438cb712c3a4e0344d05c6ebd4b7aae6...,0,0,0,1,0,2018-12-15 21:27:11.588,2018-12-15 21:27:11.588,0.000,3
3,00014c4428e61aca7e64be1064d2b6cf49233cfb,14.0,2.0,[1206216ad33f70eacc1036ac8690a1857168e9fd03512...,2.0,1.0,0.0,0.0,0.0,1206216ad33f70eacc1036ac8690a1857168e9fd035127...,0,1,0,1,0,2018-12-23 07:40:11.488,2018-12-23 07:40:11.500,0.012,2
4,00014c4428e61aca7e64be1064d2b6cf49233cfb,14.0,2.0,[1206216ad33f70eacc1036ac8690a1857168e9fd03512...,2.0,1.0,0.0,0.0,0.0,f916f313d54625ea5e67a58f6434ac59abbee1f4de10b6...,0,0,0,1,0,2018-12-23 07:41:50.607,2018-12-23 07:41:50.607,0.000,2


In [ ]:
# Adding 'hour_of_first_interaction' and 'day_of_week' columns
df_final['hour_of_first_interaction'] = df_final['first_interaction'].dt.hour
df_final['hour_of_last_interaction'] = df_final['last_interaction'].dt.hour



column_order_final = [
    'session_id_hash',
    'product_skus_hash',  # Keeping this second
    'seen product url',  # Moved up
    'number_of_seen_url',  # Moved up
    'pageview',
    'detail',
    'event',
    'click',
    'add',
    'purchase',
    'remove',
    'product_action_event_add',
    'product_action_event_click',
    'product_action_event_remove',
    'product_action_pageview_detail',
    'product_action_pageview_purchase',
    'first_interaction',
    'last_interaction',
    'engagement_duration',
    'hour_of_first_interaction',
    'hour_of_last_interaction',
    'day_of_week'
]

# Reordering df_final according to the final column order
df_final = df_final[column_order_final]

df_final.head()


,session_id_hash,product_skus_hash,seen product url,number_of_seen_url,pageview,detail,event,click,add,purchase,...,product_action_event_click,product_action_event_remove,product_action_pageview_detail,product_action_pageview_purchase,first_interaction,last_interaction,engagement_duration,hour_of_first_interaction,hour_of_last_interaction,day_of_week
0,00007d15aeb741b3cdd873cb3933351d699cc320,90add4dd072e160034e98ddea257e0b59441eae00d8955...,[90add4dd072e160034e98ddea257e0b59441eae00d895...,3,7.0,3.0,0.0,0.0,0.0,0.0,...,0,0,1,0,2018-12-15 21:23:55.879,2018-12-15 21:23:55.879,0.000,21,21,Saturday
1,00007d15aeb741b3cdd873cb3933351d699cc320,9fe0ebc298344bafad714091892d02a5a501f1bffe9fa5...,[90add4dd072e160034e98ddea257e0b59441eae00d895...,3,7.0,3.0,0.0,0.0,0.0,0.0,...,0,0,1,0,2018-12-15 21:28:54.734,2018-12-15 21:28:54.734,0.000,21,21,Saturday
2,00007d15aeb741b3cdd873cb3933351d699cc320,bd81fd361f0ae5438cb712c3a4e0344d05c6ebd4b7aae6...,[90add4dd072e160034e98ddea257e0b59441eae00d895...,3,7.0,3.0,0.0,0.0,0.0,0.0,...,0,0,1,0,2018-12-15 21:27:11.588,2018-12-15 21:27:11.588,0.000,21,21,Saturday
3,00014c4428e61aca7e64be1064d2b6cf49233cfb,1206216ad33f70eacc1036ac8690a1857168e9fd035127...,[1206216ad33f70eacc1036ac8690a1857168e9fd03512...,2,14.0,2.0,2.0,1.0,0.0,0.0,...,1,0,1,0,2018-12-23 07:40:11.488,2018-12-23 07:40:11.500,0.012,7,7,Sunday
4,00014c4428e61aca7e64be1064d2b6cf49233cfb,f916f313d54625ea5e67a58f6434ac59abbee1f4de10b6...,[1206216ad33f70eacc1036ac8690a1857168e9fd03512...,2,14.0,2.0,2.0,1.0,0.0,0.0,...,0,0,1,0,2018-12-23 07:41:50.607,2018-12-23 07:41:50.607,0.000,7,7,Sunday


In [ ]:
df_final.to_csv('df_final.csv', index=False)

In [ ]:
sample = df_final.sample(n=100, random_state=42)
sample.to_csv('sample.csv', index=False)

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520732 entries, 0 to 1520731
Data columns (total 22 columns):
 #   Column                            Non-Null Count    Dtype         
---  ------                            --------------    -----         
 0   session_id_hash                   1520732 non-null  object        
 1   product_skus_hash                 1520732 non-null  object        
 2   seen product url                  1520732 non-null  object        
 3   number_of_seen_url                1520732 non-null  int64         
 4   pageview                          1520732 non-null  float64       
 5   detail                            1520732 non-null  float64       
 6   event                             1520732 non-null  float64       
 7   click                             1520732 non-null  float64       
 8   add                               1520732 non-null  float64       
 9   purchase                          1520732 non-null  float64       
 10  remove            

In [ ]:
df_final.drop(columns=['pageview', 'detail', 'event', 'click', 'add', 'purchase', 'remove'], axis=1, inplace=True)